In [7]:
#1. setup and loading 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#defining file path for dataset
file_path = '/Volumes/SHAYNE SSD/Project_Datasets/House_Price_Predictor/data_kaggle.csv'

#attempt to load data into dataframe 'df'
df = pd.read_csv(file_path)
# print("Dataset loaded succesfully. First 55 rows:")
# df.head()

In [10]:
#2. Initial Data Exploration
print("Dataframe info:")
df.info() #tells us the number of entries, the column names, the number of non-null values, and the data type of each column.

print("\nDataframe shape:")
print(f"rows: {df.shape[0]} \n columns: {df.shape[1]}")

Dataframe info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53883 entries, 0 to 53882
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Location       53883 non-null  object 
 1   Price          53635 non-null  object 
 2   Rooms          52177 non-null  object 
 3   Bathrooms      51870 non-null  float64
 4   Car Parks      36316 non-null  float64
 5   Property Type  53858 non-null  object 
 6   Size           52820 non-null  object 
 7   Furnishing     46953 non-null  object 
dtypes: float64(2), object(6)
memory usage: 3.3+ MB

Dataframe shape:
rows: 53883 
 columns: 8


In [16]:
#3. Data cleaning
print("before clean:")
print(df["Price"].head())
print("\ndata type before clean:")
print(df["Price"].dtype)

df["Price_Original"] = df["Price"] #create copy of 'Price' for backup 
df["Price"] = df["Price"].str.replace("RM", "").str.replace(",", "") #replace "RM" and "," with blank (remove)

df["Price"] = pd.to_numeric(df["Price"], errors = 'coerce')

#check
print("\n\nafter clean:")
print(df["Price"].head())
print("\ndata type after clean:")
print(df["Price"].dtype)

before clean:
0     1250000
1     6800000
2     1030000
3         NaN
4      900000
Name: Price, dtype: object

data type before clean:
object


after clean:
0    1250000.0
1    6800000.0
2    1030000.0
3          NaN
4     900000.0
Name: Price, dtype: float64

data type after clean:
float64
